In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import (StructType, StructField, StringType,
                               DoubleType, IntegerType, LongType,DateType)
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark import SparkConf
from pyspark import SparkContext
import multiprocessing
from datetime import datetime
import numpy as np
import pandas as pd
import csv

In [23]:
datedictAS = np.load('AvSigVersionTimestamps.npy')
datedictOS = np.load('OSVersionTimestamps.npy')

In [56]:
rango_av = len(datedictAS[()])
rango_os = len(datedictOS[()])

In [57]:
df_os = pd.DataFrame(datedictOS.tolist(),index=range(1,rango_os+1))
df_av = pd.DataFrame(datedictAS.tolist(),index=range(1,rango_av+1))

In [58]:
data_os = df_os.transpose().reset_index()
data_av = df_av.transpose().reset_index()

In [59]:
data_os = data_os.iloc[:,:2]
data_av = data_av.iloc[:,:2]

In [60]:
data_os.columns = ['Census_OSVersion', 'DateCensus_OSVersion']
data_av.columns = ['AvSigVersion', 'DateAvSigVersion']

In [63]:
print(data_os.head())
print(data_av.head())

   Census_OSVersion DateCensus_OSVersion
0    10.0.14393.351           2016-10-27
1   10.0.14393.2097           2018-02-22
2  10.0.10240.17918           2018-07-16
3   10.0.17744.1004           2018-09-07
4    10.0.16299.665           2018-09-11
   AvSigVersion    DateAvSigVersion
0   1.155.266.0 2013-07-18 09:08:00
1   1.167.387.0 2014-02-21 14:20:00
2  1.169.1625.0 2014-04-03 01:17:00
3  1.169.2478.0 2014-04-12 17:16:00
4    1.169.55.0 2014-03-18 01:09:00


In [64]:
data_os.to_csv("fechas_os.csv",index=False)
data_av.to_csv("fechas_av.csv",index=False)

In [2]:
conf = SparkConf()
cores = multiprocessing.cpu_count()
conf = SparkConf()
conf.set("spark.sql.shuffle.partitions", int(8))
conf.set("spark.default.parallelism", int(8))
sc = SparkContext(conf=conf)

In [3]:
spark = SparkSession.builder.appName("Microsoft_Kaggle").getOrCreate()

In [4]:
df_num = spark.read.csv("../data/df_cat_prepro_0/*.csv",inferSchema=True,header=True).limit(5000)
df_num.persist()
df_num.count()

5000

In [5]:
df_fechas_av = spark.read.csv("fechas_av.csv",inferSchema=True,header=True)
df_fechas_os = spark.read.csv("fechas_os.csv",inferSchema=True,header=True)

In [6]:
df_fechas_os = df_fechas_os.withColumn('DateCensus_OSVersion', to_date(col('DateCensus_OSVersion')))
df_fechas_av = df_fechas_av.withColumn('DateAvSigVersion', to_date(col('DateAvSigVersion')))

In [48]:
# df_fechas_av.show(8)

In [49]:
# df_fechas_os.show(8)

In [50]:
# df_num.select('AvSigVersion').distinct().count()

In [51]:
# df_num.select('Census_OSVersion').distinct().count()

In [52]:
# df_num.select('OsBuildLab').distinct().count()

El 5 elemento de la columna OsBuildLab es la fecha de release, por lo que hay que sacarla para aportar info!!!

In [53]:
# df_num.select('OsBuildLab').distinct().show(776, truncate = False)

In [7]:
df_num = df_num.withColumn('OsBuildLab', regexp_replace('OsBuildLab', '\*', '.').cast(StringType()))

In [8]:
# df_num.select('OsBuildLab').distinct().show(776, truncate = False)

In [8]:
df_date_osbuild = df_num.withColumn('OsBuildLab_4', split(df_num['OsBuildLab'], '\.')[4].cast(StringType()))
df_date_osbuild = df_date_osbuild.withColumn('OsBuildLab_date', split(df_date_osbuild['OsBuildLab_4'], '-')[0].cast(StringType()))

In [10]:
# df_date_osbuild.select('OsBuildLab_date').show()

In [9]:
df_date_osbuild = df_date_osbuild.withColumn('OsBuildLab_date', to_date(col('OsBuildLab_date'), format='yyMMdd'))

In [56]:
# df_date_osbuild.select('OsBuildLab_date').show()

In [57]:
# a = df_date_osbuild.orderBy('AvSigVersion')

In [58]:
# a.select('AvSigVersion').distinct().count()

In [59]:
# AvSigVersion_dis = a.select('AvSigVersion').distinct().collect()

In [19]:
# AvSigVersion_dis

In [10]:
df_dates = df_date_osbuild.join(df_fechas_av, ['AvSigVersion'], 'left').select('MachineIdentifier',
                                                                               'Platform',
                                                                               'OsBuildLab_date',
                                                                               'DateAvSigVersion')

In [11]:
# df_dates.show()

In [23]:
w1 = Window.partitionBy('Platform').orderBy('OsBuildLab_date')
w2 = Window.partitionBy('Platform').orderBy('DateAvSigVersion')
w3 = Window.partitionBy().orderBy('OsBuildLab_date')
w4 = Window.partitionBy().orderBy('DateAvSigVersion')

In [13]:
data_windows = df_dates.withColumn('OsBuildLab_date_lag', lag('OsBuildLab_date').over(w1))
data_windows.persist()
data_windows.count()

In [15]:
date_diff = data_windows.withColumn('OSBuild_diff', datediff(col('OsBuildLab_date'), col('OsBuildLab_date_lag')))

In [17]:
date_diff_2 = date_diff.withColumn('DateAvSigVersion_lag', lag('DateAvSigVersion').over(w2))
date_diff_2.persist()
date_diff_2.count()

In [20]:
date_diff_3 = date_diff_2.withColumn('AvSigVersion_diff', datediff(col('DateAvSigVersion'), col('DateAvSigVersion_lag')))

In [24]:
data_windows = date_diff_3.withColumn('OsBuildLab_date_fulllag', lag('OsBuildLab_date').over(w3))
data_windows.persist()
data_windows.count()

5000

In [29]:
data_windows = data_windows.withColumn('OSBuild_fulldiff', datediff(col('OsBuildLab_date'), col('OsBuildLab_date_fulllag')))

In [25]:
data_windows = data_windows.withColumn('DateAvSigVersion_fulllag', lag('DateAvSigVersion').over(w4))
data_windows.persist()
data_windows.count()

5000

In [28]:
data_windows = data_windows.withColumn('AvSigVersion_fulldiff', datediff(col('DateAvSigVersion'), col('DateAvSigVersion_fulllag')))

In [31]:
data_windows.filter((col('AvSigVersion_diff') > 0) & (col('Platform') != 'windows10')).show(3)

+--------------------+--------+---------------+----------------+-------------------+------------+--------------------+-----------------+-----------------------+------------------------+---------------------+----------------+
|   MachineIdentifier|Platform|OsBuildLab_date|DateAvSigVersion|OsBuildLab_date_lag|OSBuild_diff|DateAvSigVersion_lag|AvSigVersion_diff|OsBuildLab_date_fulllag|DateAvSigVersion_fulllag|AvSigVersion_fulldiff|OSBuild_fulldiff|
+--------------------+--------+---------------+----------------+-------------------+------------+--------------------+-----------------+-----------------------+------------------------+---------------------+----------------+
|5357a92653a18f2d5...|windows8|     2017-04-27|      2017-05-29|         2017-02-04|          82|          2016-07-08|              325|             2017-04-27|              2017-05-22|                    7|               0|
|535fa26f687e7abdd...|windows8|     2017-09-14|      2018-01-08|         2017-09-14|           0|   

In [33]:
df_max_date = data_windows.groupBy('Platform').agg(max('OsBuildLab_date'),
                                                  max('DateAvSigVersion'),
                                                  max('OSBuild_diff'),
                                                  max('AvSigVersion_diff'),
                                                  max('OSBuild_fulldiff'),
                                                  max('AvSigVersion_fulldiff'))

In [38]:
df_date_max_date = data_windows.join(df_max_date, ['Platform'], 'left')

In [39]:
df_date_max_date.show(3)

+---------+--------------------+---------------+----------------+-------------------+------------+--------------------+-----------------+-----------------------+------------------------+---------------------+----------------+--------------------+---------------------+-----------------+----------------------+---------------------+--------------------------+
| Platform|   MachineIdentifier|OsBuildLab_date|DateAvSigVersion|OsBuildLab_date_lag|OSBuild_diff|DateAvSigVersion_lag|AvSigVersion_diff|OsBuildLab_date_fulllag|DateAvSigVersion_fulllag|AvSigVersion_fulldiff|OSBuild_fulldiff|max(OsBuildLab_date)|max(DateAvSigVersion)|max(OSBuild_diff)|max(AvSigVersion_diff)|max(OSBuild_fulldiff)|max(AvSigVersion_fulldiff)|
+---------+--------------------+---------------+----------------+-------------------+------------+--------------------+-----------------+-----------------------+------------------------+---------------------+----------------+--------------------+---------------------+--------------

In [45]:
df_date_max_date = df_date_max_date.withColumn('OsBuildLab_difftotal', datediff(col('max(OsBuildLab_date)'), col('OsBuildLab_date')))\
.withColumn('DateAvSigVersion_difftotal', datediff(col('max(DateAvSigVersion)'), col('DateAvSigVersion')))\
.withColumn('DateAvSigVersion_fulldifftotal', datediff(col('max(DateAvSigVersion)'), col('DateAvSigVersion_fulllag')))\
.withColumn('OsBuildLab_fulldifftotal', datediff(col('max(OsBuildLab_date)'), col('OsBuildLab_date_fulllag')))\
.withColumn('DateAvSigVersion_ratio', round(col('AvSigVersion_diff')/col('max(AvSigVersion_diff)')*100, 2))\
.withColumn('OsBuildLab_ratio', round(col('OSBuild_diff')/col('max(OSBuild_diff)')*100, 2))\
.withColumn('DateAvSigVersion_fullratio', round(col('AvSigVersion_fulldiff')/col('max(AvSigVersion_diff)')*100, 2))\
.withColumn('OsBuildLab_fullratio', round(col('OSBuild_fulldiff')/col('max(OSBuild_diff)')*100, 2))

In [46]:
df_date_max_date.show(3)

+---------+--------------------+---------------+----------------+-------------------+------------+--------------------+-----------------+-----------------------+------------------------+---------------------+----------------+--------------------+---------------------+-----------------+----------------------+---------------------+--------------------------+--------------------+--------------------------+------------------------------+------------------------+----------------------+----------------+--------------------------+--------------------+
| Platform|   MachineIdentifier|OsBuildLab_date|DateAvSigVersion|OsBuildLab_date_lag|OSBuild_diff|DateAvSigVersion_lag|AvSigVersion_diff|OsBuildLab_date_fulllag|DateAvSigVersion_fulllag|AvSigVersion_fulldiff|OSBuild_fulldiff|max(OsBuildLab_date)|max(DateAvSigVersion)|max(OSBuild_diff)|max(AvSigVersion_diff)|max(OSBuild_fulldiff)|max(AvSigVersion_fulldiff)|OsBuildLab_difftotal|DateAvSigVersion_difftotal|DateAvSigVersion_fulldifftotal|OsBuildLab_f

In [49]:
final_dates = df_date_max_date.withColumn('OsBuildLab_dayOfWeek', date_format('OsBuildLab_date', 'u'))\
.withColumn('AvSigVersion_dayOfWeek', date_format('DateAvSigVersion', 'u'))

In [50]:
final_dates.filter(col('OsBuildLab_dayOfWeek')==1).show(3)

+---------+--------------------+---------------+----------------+-------------------+------------+--------------------+-----------------+-----------------------+------------------------+---------------------+----------------+--------------------+---------------------+-----------------+----------------------+---------------------+--------------------------+--------------------+--------------------------+------------------------------+------------------------+----------------------+----------------+--------------------------+--------------------+--------------------+----------------------+
| Platform|   MachineIdentifier|OsBuildLab_date|DateAvSigVersion|OsBuildLab_date_lag|OSBuild_diff|DateAvSigVersion_lag|AvSigVersion_diff|OsBuildLab_date_fulllag|DateAvSigVersion_fulllag|AvSigVersion_fulldiff|OSBuild_fulldiff|max(OsBuildLab_date)|max(DateAvSigVersion)|max(OSBuild_diff)|max(AvSigVersion_diff)|max(OSBuild_fulldiff)|max(AvSigVersion_fulldiff)|OsBuildLab_difftotal|DateAvSigVersion_difftotal

In [57]:
drop_list = ['Platform', 'OsBuildLab_date', 'DateAvSigVersion', 'OsBuildLab_date_lag',
             'DateAvSigVersion_lag', 'max(OsBuildLab_date)', 'max(DateAvSigVersion)',
             'OsBuildLab_date_fulllag', 'DateAvSigVersion_fulllag', 'max(OSBuild_diff)',
             'max(AvSigVersion_diff)', 'max(OSBuild_fulldiff)', 'max(AvSigVersion_fulldiff)']

for c in drop_list:
    final_dates = final_dates.drop(c)

In [58]:
final_dates_imputed = final_dates.fillna(0)

In [59]:
final_dates_imputed.show()

+--------------------+------------+-----------------+---------------------+----------------+--------------------+--------------------------+------------------------------+------------------------+----------------------+----------------+--------------------------+--------------------+--------------------+----------------------+
|   MachineIdentifier|OSBuild_diff|AvSigVersion_diff|AvSigVersion_fulldiff|OSBuild_fulldiff|OsBuildLab_difftotal|DateAvSigVersion_difftotal|DateAvSigVersion_fulldifftotal|OsBuildLab_fulldifftotal|DateAvSigVersion_ratio|OsBuildLab_ratio|DateAvSigVersion_fullratio|OsBuildLab_fullratio|OsBuildLab_dayOfWeek|AvSigVersion_dayOfWeek|
+--------------------+------------+-----------------+---------------------+----------------+--------------------+--------------------------+------------------------------+------------------------+----------------------+----------------+--------------------------+--------------------+--------------------+----------------------+
|5363bedd7963